In [82]:
from pathlib import Path
import pandas as pd
import geopandas as gpd

import shapely

These are downloaded from https://geoportal.statistics.gov.uk/datasets/70a430114cf3436a84880f0969eaf283_0

https://geoportal.statistics.gov.uk/search?q=Westminster_Parliamentary_Constituencies_Dec_2022_UK_BGC
https://geoportal.statistics.gov.uk/search?q=Westminster%20Parliamentary%20Constituencies%20UK%20BGC

https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Westminster_Parliamentary_Constituencies_Dec_2022_UK_BGC/FeatureServer


https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Westminster_Parliamentary_Constituencies_Dec_2022_UK_BGC/FeatureServer/0

In [ ]:
csv_url =    'https://stg-arcgisazurecdataprod1.az.arcgis.com/exportfiles-1559-19445/Westminster_Parliamentary_Constituencies_Dec_2022_UK_BGC_-4629094885293682949.csv?sv=2018-03-28&sr=b&sig=CSlhL5mC0AaukzNQsUu1qRjfUnjbdcknlZmWGkdZV9g%3D&se=2024-02-08T01%3A40%3A26Z&sp=r'

In [ ]:
data = pd.read_csv(csv_url, usecols=['PCON22CD', 'PCON22NM'], index_col=['PCON22CD'])

In [86]:
geojson_url = 'https://stg-arcgisazurecdataprod1.az.arcgis.com/exportfiles-1559-19445/Westminster_Parliamentary_Constituencies_Dec_2022_UK_BGC_7450510434469596417.geojson?sv=2018-03-28&sr=b&sig=7aBo7Z45VoYgrWtwpJw9B%2FjIwaneZiET6PFUc1%2B9E5g%3D&se=2024-02-22T23%3A41%3A07Z&sp=r'

In [87]:
geo = (
  gpd
    .read_file(geojson_url, engine="pyogrio")
    .to_crs('EPSG:4326')
    .query('PCON22CD.str.startswith("E")')
    .loc[:, ['PCON22CD', 'PCON22NM', 'geometry']]
)
geo.geometry = shapely.set_precision(geo.geometry, grid_size=0.00001)
geo.crs = None

In [91]:
OUTPUT_DIR = Path('../../src/_data/areas/')

REF_DIR = OUTPUT_DIR / 'reference'
REF_DIR.mkdir(parents=True, exist_ok=True)

MAPS_DIR = OUTPUT_DIR / 'maps'
MAPS_DIR.mkdir(parents=True, exist_ok=True)

DATA_DIR = Path('../../src/generated/areas/constituency/_data/')
DATA_DIR.mkdir(parents=True, exist_ok=True)

In [92]:
pd.DataFrame(geo.loc[:, ['PCON22CD', 'PCON22NM']]).to_json(REF_DIR / 'pcon.json', orient='records')

In [93]:
geo.to_file(MAPS_DIR / 'pcon.geojson', engine='fiona', driver='GeoJSON')

In [ ]:
import json

with open(MAPS_DIR / 'pcon.geojson') as geojson:
    d = json.load(geojson) 

In [ ]:
res = {}
for f in d['features']:
    res[f['properties']['PCON22CD']] = {
      "type": "FeatureCollection",
      "features": [f]
    }

with open(DATA_DIR / 'map.json', 'w') as f:
    json.dump(res, f)